# Tarefa 5 - Treinando Modelos Densos Estáticos
## Alunos
- Yago César
- Samuel Morais


## Imports e logging

In [1]:
import ast
from time import time
import logging
import pandas as pd
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Lendo o corpus pré-processado

In [2]:
df = pd.read_csv("../data/TokenizedSentences.csv")

# Transformando as strings "[...strings]" para listas de strings
df['text'] = df['text'].apply(ast.literal_eval)

corpus = df['text'].tolist()
df['text']

0        [nada, queira, voltar, tempo, refazer, vida, t...
1        [lidando, mudanças, humor, tido, muitas, mudan...
2        [noiva, usar, voce, ja, sabe, pergunte, nao, f...
3        [muitas, vezes, consigo, trabalhar, cerca, hor...
4        [quais, ações, sentindo, ansioso, procurando, ...
                               ...                        
50962    [vivo, outras, fazendo, péssimo, trabalho, viv...
50963    [parabéns, amigo, tapa, buraco, depende, quer,...
50964    [ansiedade, doente, epiléptico, ansiedade, def...
50965    [passei, período, difícil, durante, últimos, 3...
50966    [sair, terapia, desde, 14, anos, finalmente, d...
Name: text, Length: 50967, dtype: object

## Modelo Word2Vec
### Treinamento

#### Opções

In [3]:
# 'skip-gram' ou 'cbow'
TRAINING_ALGORITHM = 'skip-gram'

In [4]:


w2v_model = Word2Vec(
    vector_size=300, # dimensionalidade do vetor de palavras
    window=5,        # distância máxima entre a palavra atual e a palavra predita em uma sentença
    min_count=5,     # ignora todas as palavras com uma frequência total menor que esse valor
    sample=6e-5,     # limiar para configurar quais palavras de maior frequência são aleatoriamente downsampled
    sg=1 if TRAINING_ALGORITHM == 'skip-gram' else 0 # CBOW = 0 ; skip-gram = 1
)

t = time()
w2v_model.build_vocab(corpus, progress_per=10000)
print(f'Time to build vocab: {round((time() - t) / 60, 2)}')

t = time()
# report_delay - número de segundos para esperar antes de reportar o progresso
w2v_model.train(corpus, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print(f'Time to train the model: {round((time() - t) / 60, 2)} mins')

w2v_model.save('word2vec_{TRAINING_ALGORITHM}.model')


2025-10-19 23:43:24,020 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.025>', 'datetime': '2025-10-19T23:43:24.020921', 'gensim': '4.4.0', 'python': '3.11.2 (main, Apr 28 2025, 14:11:48) [GCC 12.2.0]', 'platform': 'Linux-6.1.0-40-amd64-x86_64-with-glibc2.36', 'event': 'created'}
2025-10-19 23:43:24,021 : INFO : collecting all words and their counts
2025-10-19 23:43:24,022 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-10-19 23:43:24,110 : INFO : PROGRESS: at sentence #10000, processed 609329 words, keeping 39103 word types
2025-10-19 23:43:24,186 : INFO : PROGRESS: at sentence #20000, processed 1203381 words, keeping 56838 word types
2025-10-19 23:43:24,266 : INFO : PROGRESS: at sentence #30000, processed 1810721 words, keeping 71456 word types
2025-10-19 23:43:24,346 : INFO : PROGRESS: at sentence #40000, processed 2414286 words, keeping 84054 word types
2025-10-19 23:43:24,434 : INFO : PROGRESS: at sentence #5

Time to build vocab: 0.01


2025-10-19 23:43:25,744 : INFO : EPOCH 0 - PROGRESS: at 20.55% examples, 337932 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:43:26,745 : INFO : EPOCH 0 - PROGRESS: at 41.13% examples, 341195 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:43:27,753 : INFO : EPOCH 0 - PROGRESS: at 59.70% examples, 332863 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:43:28,756 : INFO : EPOCH 0 - PROGRESS: at 77.20% examples, 323442 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:43:29,792 : INFO : EPOCH 0 - PROGRESS: at 96.74% examples, 323353 words/s, in_qsize 6, out_qsize 0
2025-10-19 23:43:29,937 : INFO : EPOCH 0: training on 3081926 raw words (1702222 effective words) took 5.2s, 325353 effective words/s
2025-10-19 23:43:30,967 : INFO : EPOCH 1 - PROGRESS: at 20.55% examples, 341587 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:43:31,992 : INFO : EPOCH 1 - PROGRESS: at 42.02% examples, 347325 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:43:32,997 : INFO : EPOCH 1 - PROGRESS: at 61.31% examples, 34

Time to train the model: 2.59 mins


2025-10-19 23:46:00,536 : INFO : saved word2vec_{TRAINING_ALGORITHM}.model


### Exploração semântica
#### Palavras mais similares

Tomando como base o contexto do corpus, as 4 palavras escohidas foram:
- depressão;
- ansiedade;
- medo;
- suicídio.

In [5]:
w2v_model.wv.most_similar(positive=["depressão"], topn=5)

[('ansiedade', 0.7341228723526001),
 ('deprimido', 0.6133078336715698),
 ('diagnosticado', 0.60204017162323),
 ('atípica', 0.581782341003418),
 ('tdm', 0.5664227604866028)]

In [6]:
w2v_model.wv.most_similar(positive=["ansiedade"], topn=5)

[('depressão', 0.7341229319572449),
 ('pânico', 0.6757805347442627),
 ('social', 0.6715758442878723),
 ('ataques', 0.651681125164032),
 ('sudorese', 0.6240261197090149)]

In [7]:
w2v_model.wv.most_similar(positive=["medo"], topn=5)

[('posso', 0.5907377004623413),
 ('preocupado', 0.5629320740699768),
 ('tanto', 0.5490846037864685),
 ('ansiedade', 0.5379822850227356),
 ('disso', 0.523711621761322)]

In [8]:
w2v_model.wv.most_similar(positive=["suicídio"], topn=5)

[('matar', 0.6774101257324219),
 ('cometer', 0.6180763244628906),
 ('acabar', 0.5466104745864868),
 ('suicida', 0.5241466760635376),
 ('morte', 0.5172737836837769)]

#### Similaridade semântica entre palavras

Palavras escolhidas:
- ansiedade;
- depressão;
- suicídio.

In [9]:
w2v_model.wv.similarity('ansiedade', 'depressão')

0.7341229

In [10]:
w2v_model.wv.similarity('suicídio', 'ansiedade')

0.23209706

In [11]:
w2v_model.wv.similarity('depressão', 'suicídio')

0.41887885

#### Analogia semântica

Analogias escolhidas:
- **"suicídio + depressão"**: entender quais palavras entram no contexto da combinação entre **"suicídio"** e **"depressão"**.
- **"depressão + ansiedade - suicídio"**: entender o que há em comum entre **depressão** e **ansiedade** sem levar em conta desfechos extremos (**suicídio**).
- **"psicólogo + saúde"**: buscar palavras que representem o papel do **psicólogo** na melhora da **saúde** dos pacientes.

In [12]:
# suicídio + depressão
w2v_model.wv.most_similar(positive=["suicídio", "depressão"])

[('suicida', 0.6351876258850098),
 ('matar', 0.605955958366394),
 ('deprimido', 0.5976631045341492),
 ('suicidas', 0.5921911597251892),
 ('ansiedade', 0.57357257604599),
 ('vida', 0.5606616139411926),
 ('pensamentos', 0.5273422002792358),
 ('mental', 0.5263992547988892),
 ('intitular', 0.5260677933692932),
 ('desde', 0.5218965411186218)]

In [13]:
# depressão + ansiedade - suicídio
w2v_model.wv.most_similar(positive=["depressão", "ansiedade"], negative=['suicídio'])

[('social', 0.5459948778152466),
 ('avpd', 0.5377596020698547),
 ('pânico', 0.5075700879096985),
 ('prescrevi', 0.5062640905380249),
 ('diagnosticado', 0.5038084387779236),
 ('ataques', 0.5024560689926147),
 ('oxalato', 0.48530519008636475),
 ('sudorese', 0.4812137186527252),
 ('ansiedadeansiedade', 0.4810433089733124),
 ('transpiração', 0.4777202308177948)]

In [14]:
# psicólogo + saúde
w2v_model.wv.most_similar(positive=["psicólogo", "saúde"])

[('mental', 0.7027981281280518),
 ('terapeuta', 0.6133944392204285),
 ('psiquiatra', 0.6069098114967346),
 ('terapia', 0.5947378277778625),
 ('médico', 0.5824423432350159),
 ('consultar', 0.532044529914856),
 ('ajuda', 0.5313453078269958),
 ('terapeutaconselheiro', 0.5214629173278809),
 ('postinho', 0.5203996300697327),
 ('avpd', 0.5180386304855347)]

## Modelo Doc2Vec
### Treinamento

In [15]:
# Para treinar o modelo, será necessário associar uma tag/número com cada documento do corpus
documents = [TaggedDocument(words=tokens, tags=[str(i)]) for i, tokens in enumerate(corpus)]

d2v_model = Doc2Vec(
    vector_size=300,
    window=5,
    min_count=5,
    epochs=30
)

t = time()
d2v_model.build_vocab(documents, progress_per=10000)
print(f'Time to build vocab: {round((time() - t) / 60, 2)}')

t = time()
d2v_model.train(documents, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs, report_delay=1)
print(f'Time to train the model: {round((time() - t) / 60, 2)} mins')

d2v_model.save('../models/doc2vec_model.model')

2025-10-19 23:46:00,939 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d300,n5,w5,mc5,s0.001,t3>', 'datetime': '2025-10-19T23:46:00.939496', 'gensim': '4.4.0', 'python': '3.11.2 (main, Apr 28 2025, 14:11:48) [GCC 12.2.0]', 'platform': 'Linux-6.1.0-40-amd64-x86_64-with-glibc2.36', 'event': 'created'}
2025-10-19 23:46:00,940 : INFO : collecting all words and their counts
2025-10-19 23:46:00,940 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2025-10-19 23:46:01,014 : INFO : PROGRESS: at example #10000, processed 609329 words (8317964 words/s), 39103 word types, 10000 tags
2025-10-19 23:46:01,102 : INFO : PROGRESS: at example #20000, processed 1203381 words (6757543 words/s), 56838 word types, 20000 tags
2025-10-19 23:46:01,317 : INFO : PROGRESS: at example #30000, processed 1810721 words (2835873 words/s), 71456 word types, 30000 tags
2025-10-19 23:46:01,400 : INFO : PROGRESS: at example #40000, processed 2414286 words (7262662 words/s), 8

Time to build vocab: 0.02


2025-10-19 23:46:02,940 : INFO : EPOCH 0 - PROGRESS: at 29.32% examples, 853630 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:46:03,948 : INFO : EPOCH 0 - PROGRESS: at 59.70% examples, 865433 words/s, in_qsize 6, out_qsize 0
2025-10-19 23:46:04,963 : INFO : EPOCH 0 - PROGRESS: at 90.38% examples, 871166 words/s, in_qsize 6, out_qsize 0
2025-10-19 23:46:05,270 : INFO : EPOCH 0: training on 3081926 raw words (2926182 effective words) took 3.3s, 876915 effective words/s
2025-10-19 23:46:06,279 : INFO : EPOCH 1 - PROGRESS: at 29.36% examples, 852843 words/s, in_qsize 6, out_qsize 0
2025-10-19 23:46:07,294 : INFO : EPOCH 1 - PROGRESS: at 59.40% examples, 857261 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:46:08,298 : INFO : EPOCH 1 - PROGRESS: at 89.42% examples, 862543 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:46:08,648 : INFO : EPOCH 1: training on 3081926 raw words (2925844 effective words) took 3.4s, 866645 effective words/s
2025-10-19 23:46:09,652 : INFO : EPOCH 2 - PROGRESS: a

Time to train the model: 1.68 mins


### Exploração semântica
Montando queries para comparar com os documentos do corpus.

In [16]:
query = ['sentir', 'ansioso', 'trabalho']
# Infere-se um vetor de similaridade para a query utilizando o modelo.
query_vector = d2v_model.infer_vector(query)

In [17]:
similar_docs = d2v_model.dv.most_similar([query_vector], topn=5)
for tag, similarity in similar_docs:
    print(f"Documento {tag} -> Similaridade: {similarity} <<{' '.join(documents[int(tag)].words)}>>")

Documento 40274 -> Similaridade: 0.7684020400047302 <<alguma dica ansiedade trabalho poderia usar alguns conselhos>>
Documento 26034 -> Similaridade: 0.7527951002120972 <<acompanho trabalho desde início 5potssempre fãabraço>>
Documento 16062 -> Similaridade: 0.7368998527526855 <<ótima vida filhos esposa trabalho carreira resta arrependimento>>
Documento 50087 -> Similaridade: 0.7305425405502319 <<desmoronei trabalho recuperar disso>>
Documento 40747 -> Similaridade: 0.7222082614898682 <<patrão fica precarizando trabalho fuder>>
